In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPool1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

2023-07-31 16:47:21.226371: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-31 16:47:21.227965: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-31 16:47:21.262048: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-31 16:47:21.262701: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 16:47:21.771141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
!wget https://lazyprogrammer.me/course_files/spam.csv

--2023-07-31 16:47:24--  https://lazyprogrammer.me/course_files/spam.csv
Resolvendo lazyprogrammer.me (lazyprogrammer.me)... 2606:4700:3030::ac43:d5a6, 2606:4700:3031::6815:17d2, 172.67.213.166, ...
Conectando-se a lazyprogrammer.me (lazyprogrammer.me)|2606:4700:3030::ac43:d5a6|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 503663 (492K) [text/csv]
Salvando em: “spam.csv.1”

spam.csv.1          100%[===================>] 491,86K   583KB/s    em 0,8s    

2023-07-31 16:47:26 (583 KB/s) - “spam.csv.1” salvo [503663/503663]



In [3]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [5]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.columns = ['labels', 'data']

In [7]:
df['b_labels'] = df['labels'].map({'ham': 0, 'spam': 1})
Y = df['b_labels'].values
Y

array([0, 0, 1, ..., 0, 0, 0])

In [8]:
df_train, df_test, Ytrain, Ytest = train_test_split(df['data'], Y, test_size=0.33)

In [9]:
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

In [13]:
word2idx = tokenizer.word_index
V = len(word2idx)
V

7198

In [14]:
data_train = pad_sequences(sequences_train)
data_train.shape
T = data_train.shape[1]

In [15]:
data_test = pad_sequences(sequences_test, maxlen=T)
data_test.shape

(1839, 189)

In [16]:
D = 20

i = Input(shape=(T,))
x = Embedding(V + 1, D)(i)
x = Conv1D(32, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPool1D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)

In [17]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [18]:
r = model.fit(data_train, Ytrain, epochs=5, validation_data=(data_test, Ytest))

Epoch 1/5
117/117 [==============================] - 1s 6ms/step - loss: 0.3811 - accuracy: 0.8653 - val_loss: 0.2771 - val_accuracy: 0.8635
Epoch 2/5
117/117 [==============================] - 0s 4ms/step - loss: 0.1345 - accuracy: 0.9397 - val_loss: 0.0893 - val_accuracy: 0.9755
Epoch 3/5
117/117 [==============================] - 0s 4ms/step - loss: 0.0244 - accuracy: 0.9930 - val_loss: 0.0684 - val_accuracy: 0.9826
Epoch 4/5
117/117 [==============================] - 0s 4ms/step - loss: 0.0062 - accuracy: 0.9989 - val_loss: 0.0953 - val_accuracy: 0.9842
Epoch 5/5
117/117 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 0.9995 - val_loss: 0.0868 - val_accuracy: 0.9837
